## Análisis de los arrepentidos

In [1]:
import pandas as pd

In [12]:
# Cargo las columnas que necesito
datos = pd.read_csv('/home/lautaro/Desktop/events.csv',
                    usecols = lambda col: col in ["event", "timestamp", "person","sku","model"], low_memory=False)

In [13]:
checkouts = datos.loc[datos["event"] == "checkout",["timestamp","person","sku","model"]]
ventas = datos.loc[datos["event"] == "conversion",["timestamp", "person", "sku", "model"]]

In [14]:
checkouts = checkouts.rename(index=str, columns={"timestamp" : "fecha_checkout"})
ventas = ventas.rename(index=str, columns={"timestamp" : "fecha_venta"})

In [21]:
# Me quedo con un unico checkout por combinacion de persona y producto
checkouts = checkouts.groupby(["person", "sku"], as_index=False).head(1)

In [32]:
merged = checkouts.merge(ventas, how="left", left_on=["person", "sku", "model"], right_on=["person", "sku", "model"])

In [37]:
# Obtengo los checkouts sin ventas
checkouts_sin_ventas = merged.loc[merged["fecha_venta"].isnull(),["sku", "model"]]
checkouts_sin_ventas["cantidad"] = 1

In [38]:
checkouts_sin_ventas = checkouts_sin_ventas.groupby(["sku", "model"], as_index=False).sum()

In [41]:
print(checkouts_sin_ventas.nlargest(10, "cantidad"))

          sku                   model  cantidad
1301   6371.0       Samsung Galaxy J5       411
1295   6357.0       Samsung Galaxy J5       405
896    2831.0                iPhone 6       310
1316   6413.0       Samsung Galaxy J7       275
894    2830.0                iPhone 6       246
1264   6023.0               iPhone 5s       230
1740   8443.0       Samsung Galaxy S7       229
890    2829.0                iPhone 6       219
445   12758.0       Samsung Galaxy S8       216
1085   3371.0  Samsung Galaxy S6 Flat       206


In [42]:
checkouts_sin_ventas = checkouts_sin_ventas.drop(columns="sku")

In [44]:
print (checkouts_sin_ventas.groupby("model", as_index=False).sum().nlargest(10, "cantidad"))

                      model  cantidad
171                iPhone 6      2981
170               iPhone 5s      2368
173               iPhone 6S      2130
85        Samsung Galaxy J5      1769
120       Samsung Galaxy S7      1181
175                iPhone 7      1143
122       Samsung Galaxy S8      1042
176           iPhone 7 Plus       869
121  Samsung Galaxy S7 Edge       785
119  Samsung Galaxy S6 Flat       782
